In [1]:
#TODO: increase size
#TODO: change train to include labels, ignore for ae

In [17]:
import tensorflow as tf
import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Dense, Conv2D, Conv2DTranspose , LeakyReLU, Dropout, Flatten, Input, Reshape

In [3]:
#run for no gpu if needed
tf.config.set_visible_devices([], 'GPU')

DATA_DIR='/home/evan/Datasets/tensorflow'
BATCH_SIZE=32
BUFFER_SIZE = 10000
IMG_SIZE = (64,64)
EMBEDDED_DIM = 32

train_dataset = tfds.load('stanford_dogs', as_supervised=True, split='train', data_dir=DATA_DIR, download=False)
test_dataset = tfds.load('stanford_dogs', as_supervised=True, split='test', data_dir=DATA_DIR, download=False)

def transform(x , y):
    x = tf.image.resize(x, IMG_SIZE)
    x = tf.cast(x, tf.float32)
    x = (x - 127.5)/127.5
    return x, y
    
# train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.uint8), y))

def transform_ae(x,y):
    return (x,x,y)

def prepare_transformed(dataset):
    return dataset.repeat().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = test_dataset.map(lambda x, y : transform(x,y))
train_dataset = train_dataset.map(lambda x, y : transform(x,y))
ae_dataset = train_dataset.map(lambda x, y : transform_ae(x,y))
ae_dataset = prepare_transformed(ae_dataset)
train_dataset = prepare_transformed(train_dataset)

print(train_dataset)
print(ae_dataset)
print(test_dataset)

<PrefetchDataset shapes: ((None, 64, 64, 3), (None,)), types: (tf.float32, tf.int64)>
<PrefetchDataset shapes: ((None, 64, 64, 3), (None, 64, 64, 3), (None,)), types: (tf.float32, tf.float32, tf.int64)>
<MapDataset shapes: ((64, 64, 3), ()), types: (tf.float32, tf.int64)>


2021-10-26 13:27:03.851961: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-26 13:27:03.857524: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-26 13:27:03.857713: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-26 13:27:03.861524: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [92]:
class Conv2DTransposeTied(Layer):
    def __init__(self, conv2D, activation=None, **kwargs):
        self.conv2D = conv2D
        self.activation = keras.activations.get(activation)
        super().__init__(**kwargs)
    
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name='bias', initializer ='zeros',
                                      shape= self.conv2D.input_shape[1:]) #exclude batch dim
    
    def call(self, inputs):
        print(inputs)
        print(self.conv2D.input_shape)
        Z = tf.nn.conv2d_transpose(inputs, self.conv2D.kernel, (BATCH_SIZE,) + self.conv2D.input_shape[1:], self.conv2D.strides, padding=self.conv2D.padding.upper())
        return self.activation(Z + self.biases)

Conv2DTransposeTied(c3)(Z)

Tensor("Placeholder:0", shape=(None, 16, 16, 128), dtype=float32)
(None, 32, 32, 128)


<KerasTensor: shape=(None, 32, 32, 128) dtype=float32 (created by layer 'conv2d_transpose_tied_41')>

In [93]:
encoder_input_img = Input(IMG_SIZE+((3,)))
encoder_input_label = Input((1))
c1 = Conv2D(64, (5, 5), strides=(1, 1), padding='same', input_shape=IMG_SIZE+((3,)))
Z = c1(encoder_input_img)
Z = LeakyReLU()(Z)
Z = Dropout(0.3)(Z)

c2 = Conv2D(128, (5, 5), strides=(2, 2), padding='same')
Z = c2(Z)
Z = LeakyReLU()(Z)
Z = Dropout(0.3)(Z)

c3 = Conv2D(128, (5, 5), strides=(2, 2), padding='same')
Z = c3(Z)
Z = LeakyReLU()(Z)
Z = Dropout(0.3)(Z)
print(Z)
Z = Flatten()(Z)
latent_embedding = Dense(EMBEDDED_DIM)(Z)

KerasTensor(type_spec=TensorSpec(shape=(None, 16, 16, 128), dtype=tf.float32, name=None), name='dropout_50/Identity:0', description="created by layer 'dropout_50'")


In [94]:
Conv2DTransposeTied(c3)

In [95]:
Z = Dense(16*16*128)(latent_embedding)
Z = Reshape((16,16,128))(Z)


c3T = Conv2DTransposeTied(c3)(Z)
c3Ta = LeakyReLU()(c3T)
c3Td = Dropout(0.3)(c3Ta)

c2T = Conv2DTransposeTied(c2)(c3Td)
c2Ta = LeakyReLU()(c2T)
c2Td = Dropout(0.3)(c2Ta)

c1T = Conv2DTransposeTied(c1)(c2T)
c1Ta = LeakyReLU()(c1T)
c1Td = Dropout(0.3)(c1Ta)

decoder_output = c1Td

autoencoder = Model(encoder_input_img, decoder_output)
autoencoder.output

Tensor("Placeholder:0", shape=(None, 16, 16, 128), dtype=float32)
(None, 32, 32, 128)
Tensor("Placeholder:0", shape=(None, 32, 32, 128), dtype=float32)
(None, 64, 64, 64)
Tensor("Placeholder:0", shape=(None, 64, 64, 64), dtype=float32)
(None, 64, 64, 3)


<KerasTensor: shape=(None, 64, 64, 3) dtype=float32 (created by layer 'dropout_53')>

In [96]:
optimizer=tf.keras.optimizers.Adam(lr=.001)

autoencoder.compile(optimizer=optimizer, loss='mse')

/home/evan/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [97]:
autoencoder.fit(ae_dataset, epochs=100, steps_per_epoch=100)

Epoch 1/100
Tensor("model/reshape_2/Reshape:0", shape=(None, 16, 16, 128), dtype=float32)
(None, 32, 32, 128)
Tensor("model/dropout_51/dropout/Mul_1:0", shape=(None, 32, 32, 128), dtype=float32)
(None, 64, 64, 64)
Tensor("model/conv2d_transpose_tied_44/add:0", shape=(None, 64, 64, 64), dtype=float32)
(None, 64, 64, 3)
Tensor("model/reshape_2/Reshape:0", shape=(None, 16, 16, 128), dtype=float32)
(None, 32, 32, 128)
Tensor("model/dropout_51/dropout/Mul_1:0", shape=(None, 32, 32, 128), dtype=float32)
(None, 64, 64, 64)
Tensor("model/conv2d_transpose_tied_44/add:0", shape=(None, 64, 64, 64), dtype=float32)
(None, 64, 64, 3)


2021-10-26 14:20:52.180258: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


 95/100 [===========================>..] - ETA: 2s - loss: 18.4457

In [ ]:
# autoencoder.save('./autoencoder')

In [ ]:
import keras
autoencoder = keras.models.load_model('/home/evan/Desktop/Convolutional Random Forest/autoencoder')

In [ ]:
predictions = autoencoder.predict(train_dataset.take(1))
fig = plt.figure(figsize=(8,4))

for i in range(predictions.shape[0]):
    plt.subplot(4, 8, i+1)
    plt.imshow(((predictions[i, :, :, :]*127.5)+127.5)/255.)
    plt.axis('off')

In [ ]:
rf_dataset = train_dataset.map(lambda x,y : ((x,y),y))
encoder = keras.Model([encoder_input_img,encoder_input_label],[encoder_output_img, encoder_input_label])
print(rf_dataset)
rf_dataset = encoder.predict(rf_dataset, steps=1000) 

In [ ]:
rf_dataset[0].shape
rf_dataset[1].shape

In [ ]:
rf_image_dataset = tf.data.Dataset.from_tensor_slices(rf_dataset[0])
rf_label_dataset = tf.data.Dataset.from_tensor_slices(rf_dataset[1])
rf_combined_dataset = tf.data.Dataset.zip(((rf_image_dataset),(rf_label_dataset)))

In [ ]:
rf_combined_dataset

In [ ]:
import tensorflow_decision_forests as tfdf
from tensorflow.keras import Sequential
import keras

#run encoder to get embeddings
# (encoder_output)
rf_model = tfdf.keras.RandomForestModel()
rf_model.fit(rf_dataset[0],rf_dataset[1])

In [ ]:
def run_inference(rf_model, encoder, input):
    embeddings = encoder.predict(input,tf.ones(input.shape[0]))
    return rf_model.predict(embeddings)
run_inference(rf_model, encoder, rf_dataset[0])